### Load preprocessed data

Run the script that downloads and processes the MovieLens data

In [ ]:
#!python ../src/download.py

In [1]:
import numpy as np
fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(train_x)
print(' ')
print(train_y)

[[   1 1193   11   10]
 [   2 1193    8   16]
 [  12 1193   15   12]
 ...
 [5780 2845    3   17]
 [5851 3607   13   20]
 [5938 2909   30    1]]
 
[[5.]
 [5.]
 [4.]
 ...
 [1.]
 [5.]
 [4.]]


### Define the MF Model

In [2]:
from abstract_model import AbstractModel

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_bias = c_bias
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def likelihood(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse, {"mse": loss_mse}
    
    def prior(self):
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        log = {"prior_user": prior_user, "prior_item": prior_item}
        total = prior_user + prior_item
        return total, log

In [4]:
from pytorch_lightning.logging import WandbLogger

k = 5
c_vector = 1e-5
model = MF(n_user, n_item, k=k, c_vector=c_vector,
          batch_size=1024)
model.save_data(train_x, train_y, test_x, test_y)

# add a logger
logger = WandbLogger(name="01_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     early_stop_callback=True,
                     gpus=1,
                     progress_bar_refresh_rate=1) 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model)

In [22]:
results = trainer.test(model)
mse = results['avg_test_loss']
rmse = np.sqrt(mse)
rmse

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(0.7624, device='cuda:0'),
 'val_loss': tensor(0.7624, device='cuda:0')}
--------------------------------------------------------------------------------



0.8731724522884321